In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('data.csv')
df = df[0:2000]

In [3]:
df['kind'].unique()

array(['movie', 'tv series', 'tv mini series', 'tv movie', 'video movie'],
      dtype=object)

In [4]:
df.columns

Index(['title', 'year', 'rating', 'director', 'cast', 'genre', 'plot',
       'language', 'budget', 'gross', 'runtime', 'akas',
       'animation_department', 'art_department', 'art_directors',
       'aspect_ratio', 'assistant_directors', 'box_office',
       'camera_department', 'canonical_title', 'casting_department',
       'casting_directors', 'certificates', 'cinematographers', 'color_info',
       'composers', 'costume_departmen', 'costume_designers', 'countries',
       'country_codes', 'cover_url', 'distributors', 'editorial_department',
       'editors', 'full_size_cover_url', 'kind', 'language_codes',
       'location_management', 'long_imdb_canonical_title', 'long_imdb_title',
       'make_up_department', 'miscellaneous', 'music_department',
       'original_air_date', 'other_companies', 'plot_outline', 'producers',
       'production_companies', 'production_designers', 'production_managers',
       'script_department', 'set_decorators', 'smart_canonical_title',
       'sma

In [5]:
# df['genre'] = df['genre'].apply(apply_genre)
# mlist = []
# for i in range(len(df)):
#     mlist.extend(df.loc[i,'genre'])
# list(set(mlist))
# y = pd.get_dummies(df.genre, prefix = 'Genre')


In [6]:
def get_director_id(s):
    start = find(s, '<Person id:')
    end = find(s, '[http]')
    id_list = []
    for i in range(len(end)):
        id_list.append(s[start[i]+11:end[i]])
    return id_list

def get_director_name(s):
    start = find(s, 'name:_')
    end = find(s, '_>')
    name_list = []
    for i in range(len(end)):
        name_list.append(s[start[i]+6:end[i]])
    return name_list
    
def find(s, ch):
    l = []
    i = s.find(ch)
    while i!= -1:
        l.append(i)
        i = s.find(ch,i+1)
    return l

def get_cast_ids(s):
    start = find(s, '<Person id:')
    end = find(s, '[http]')
    id_list = []
    for i in range(len(end)):
        id_list.append(s[start[i]+11:end[i]])
    return id_list

def get_cast_names(s):
    start = find(s, 'name:_')
    end = find(s, '_>')
    name_list = []
    for i in range(len(end)):
        name_list.append(s[start[i]+6:end[i]])
    return name_list

def get_budget(s):
    start = s.find('$')
    end = s.find(' ')
    return int(s[start+1:end].replace(',',''))

def apply_genre(s):
    s = s.replace('\'','').replace('[','').replace(']','').split(sep = ', ')
    return s

def get_date(s):
    day = s[0:2]
    month = s[3:6]
    year = s[7:11]
    return day, month, year

def get_country(s):
    start = find(s, '(')
    end = find(s, ')')
    return s[start[0]+1:end[0]]

def get_runtime(s):
    s2 = s[2:]
    s3 = s2[:len(s2)-2]
    return int(s3)

In [7]:
director_name_list = []
director_id_list = []
cast_name_list = []
cast_id_list = []


for i in df.index:
    s = df.loc[i,'director']
    
    try:
        director_name_list.append(get_director_name(s))
    except:
        director_name_list.append(np.nan)

    try:
        director_id_list.append(get_director_id(s))
    except:
        director_id_list.append(np.nan)
        
    s = df.loc[i,'cast']
    
    try:
        cast_name_list.append(get_cast_names(s))
    except:
        cast_name_list.append(np.nan)
            
    try:
        cast_id_list.append(get_cast_ids(s))
    except:
        cast_id_list.append(np.nan)
    
    s = df.loc[i,'year']
    try:
        df['year'][i] = int(s)
    except:
        df['year'][i] = np.nan

for i in df.index:
    s = df.loc[i,'original_air_date']
    # print(s)
    try:
        df.loc[i,'country'] = get_country(s)
        d,m,y = get_date(s)
        df.loc[i,'day'] = d 
        df.loc[i,'month'] = m
    except:
        df.loc[i,'country'] = np.nan
        df.loc[i,'day'] = np.nan
        df.loc[i,'month'] = np.nan

for i in df.index:
    try:
        s = df.loc[i, 'runtime']
        df.loc[i, 'runtime'] = get_runtime(s)
    except:
        s = np.nan

for i in df.index:
    try:
        s = df.loc[i, 'color_info']
        if "Black" in s:
            df.loc[i, 'color_info'] = "Black and White"
        if "Color" in s:
            df.loc[i, 'color_info'] = "Color"
    except:
        s = np.nan

for i in df.index:
    try:
        s = df.loc[i, 'sound_mix']
        if "Silent" in s:
            df.loc[i, 'sound_mix'] = "Silent"
        elif "Mono" in s:
            df.loc[i, 'sound_mix'] = "Mono"
        elif "Dolby" in s:
            df.loc[i, 'sound_mix'] = "Dolby"
        elif "DTS" in s:
            df.loc[i, 'sound_mix'] = "DTS"
        else:
            df.loc[i, 'sound_mix'] = "Others"
    except:
        s = np.nan
        


<ipython-input-7-ed49b766b22c>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'][i] = int(s)


In [8]:
df['director_id'] = director_id_list
df['director_name'] = director_name_list
df['cast_ids'] = cast_id_list
df['cast_names'] = cast_name_list


In [9]:
df = df[['title','year','rating','runtime','kind','color_info','sound_mix','director_name', 'genre','director_id', 'cast_names', 
         'cast_ids','votes','country','day','month']]

In [10]:
#drop các dòng trắng trơn
df.dropna(subset = ["rating"], inplace=True)


In [11]:
month_dict = {'Oct':8, 'Dec':12, 'Aug':8, 'Jul':7, 'Feb':2, 'Nov':11, 'Sep':10, 'May':5,'Mar':3, 'Jan':1, 'Jun':6, 'Apr':3}
day_list = []
for i in range(1,32):
    day_list.append(str(i))
df.month = df.month.map(month_dict).fillna(0)
df['day'] = df.day.apply(lambda s: int(s) if s in day_list else 0)

In [12]:
df

,title,year,rating,runtime,kind,color_info,sound_mix,director_name,genre,director_id,cast_names,cast_ids,votes,country,day,month
0,Miss Jerry,1894,5.9,40,movie,Black and White,Silent,[Alexander Black],"['Short', 'Romance']",[0085156],"[Blanche Bayliss, William Courtenay, Chauncey ...","[0063086, 0183823, 1309758]",191.0,USA,0,8.0
1,The Story of the Kelly Gang,1906,6.1,70,movie,Black and White,Silent,[Charles Tait],"['Action', 'Adventure', 'Biography', 'Crime', ...",[0846879],"[Elizabeth Tait, John Tait, Nicholas Brierley,...","[0846887, 0846894, 1431224, 3002376, 0143899, ...",737.0,Australia,26,12.0
2,Den sorte drøm,1911,5.9,53,movie,Black and White,Silent,[Urban Gad],['Drama'],[0300487],"[Asta Nielsen, Valdemar Psilander, Gunnar Hels...","[0003425, 0699637, 0375839, 0016799, 0282348, ...",218.0,Germany,19,8.0
3,Cleopatra,1963,7.0,192,movie,Color,DTS,[Joseph L. Mankiewicz],"['Biography', 'Drama', 'History', 'Romance']",[0000581],"[Elizabeth Taylor, Richard Burton, Rex Harriso...","[0000072, 0000009, 0001322, 0114386, 0170552, ...",31526.0,Canada,31,7.0
4,Torment,1994,7.1,102,movie,Color,Dolby,[Claude Chabrol],"['Crime', 'Drama', 'Thriller']",[0001031],"[Emmanuelle Béart, François Cluzet, Nathalie C...","[0000322, 0167388, 0136597, 0932870, 0492200, ...",4936.0,France,16,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Resurrection,2001,6.9,187,tv movie,Color,Others,"[Paolo Taviani, Vittorio Taviani]","['Drama', 'History']","[0851752, 0851754]","[Stefania Rocca, Timothy Peach, Marie Bäumer, ...","[0733625, 0668742, 0126669, 0092291, 0160019, ...",214.0,France,26,12.0
1996,We're Not Dressing,1934,6.1,74,movie,Black and White,Mono,[Norman Taurog],"['Comedy', 'Musical']",[0851537],"[Bing Crosby, Carole Lombard, George Burns, Gr...","[0001078, 0001479, 0122675, 0020555, 0581062, ...",775.0,USA,27,3.0
1997,We're Rich Again,1934,6.4,71,movie,Black and White,Mono,[William A. Seiter],['Comedy'],[0782682],"[Edna May Oliver, Billie Burke, Marian Nixon, ...","[0646829, 0000992, 0633260, 0219666, 0550583, ...",230.0,USA,13,7.0
1998,Family Life,1971,7.6,108,movie,Color,Mono,[Ken Loach],['Drama'],[0516360],"[Sandy Ratcliff, Bill Dean, Grace Cave, Malcol...","[0711605, 0212674, 0147016, 0862939, 0552392, ...",1597.0,Ireland,18,2.0


In [13]:
df.to_csv('preprocess_data.csv', index =False)